In [2]:
import os
import pandas as pd

# Path to your folder containing CSV files
folder_path = "C:/Projects/final project/data"

# List to store column names from all CSVs
all_columns = set()

# Loop through all CSV files in the folder
for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_csv(file_path)
            print(f"File: {file}")
            print("Columns:", list(df.columns))
            print("-" * 50)
        except Exception as e:
            print(f"Could not read {file}: {e}")


File: blinkit_customers.csv
Columns: ['customer_id', 'customer_name', 'email', 'phone', 'address', 'area', 'pincode', 'registration_date', 'customer_segment', 'total_orders', 'avg_order_value']
--------------------------------------------------
File: blinkit_customer_feedback.csv
Columns: ['feedback_id', 'order_id', 'customer_id', 'rating', 'feedback_text', 'feedback_category', 'sentiment', 'feedback_date']
--------------------------------------------------
File: blinkit_marketing_performance.csv
Columns: ['campaign_id', 'campaign_name', 'date', 'target_audience', 'channel', 'impressions', 'clicks', 'conversions', 'spend', 'revenue_generated', 'roas']
--------------------------------------------------
File: blinkit_orders.csv
Columns: ['order_id', 'customer_id', 'order_date', 'promised_delivery_time', 'actual_delivery_time', 'delivery_status', 'order_total', 'payment_method', 'delivery_partner_id', 'store_id']
--------------------------------------------------
File: blinkit_order_items

In [2]:
import pandas as pd
import psycopg2
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score

# -------------------------
# LOAD DATA
# -------------------------
conn = psycopg2.connect(
    host="localhost",
    database="blinkit_db",
    user="postgres",
    password="2741"
)

query = """
SELECT
    order_id,
    order_date,
    hour_of_day,
    day_of_week,
    is_weekend,
    area,
    order_total,
    total_items,
    promised_duration_minutes,
    is_late
FROM ml_delivery_features
ORDER BY order_date;
"""

df = pd.read_sql(query, conn)
conn.close()

df = df.dropna()

X = df.drop(columns=["order_id", "order_date", "is_late"])
y = df["is_late"]

# -------------------------
# TIME-AWARE SPLIT
# -------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, shuffle=False
)

# -------------------------
# PREPROCESSING
# -------------------------
categorical_cols = ["area"]
numerical_cols = [
    "hour_of_day",
    "day_of_week",
    "is_weekend",
    "order_total",
    "total_items",
    "promised_duration_minutes"
]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_cols),
        ("num", "passthrough", numerical_cols)
    ]
)

# -------------------------
# MODEL
# -------------------------
model = HistGradientBoostingClassifier(
    max_depth=6,
    learning_rate=0.08,
    max_iter=250,
    random_state=42
)

pipeline = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", model)
])

# -------------------------
# TRAIN
# -------------------------
pipeline.fit(X_train, y_train)

# -------------------------
# EVALUATE
# -------------------------
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)

print("HistGradientBoosting AUC:", auc)


C:\Users\Tasneem Inayath\AppData\Local\Temp\ipykernel_10868\3674095147.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


HistGradientBoosting AUC: 0.4961950169437368


In [9]:
import pandas as pd
import psycopg2

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# -----------------------------
# LOAD DATA FROM POSTGRES
# -----------------------------
conn = psycopg2.connect(
    host="localhost",
    database="blinkit_db",
    user="postgres",
    password="2741"
)

query = """
SELECT
    order_id,
    order_date,
    hour_of_day,
    day_of_week,
    is_weekend,
    area,
    order_total,
    total_items,
    promised_duration_minutes,
    is_late
FROM ml_delivery_features
ORDER BY order_date;
"""

df = pd.read_sql(query, conn)
conn.close()

# -----------------------------
# BASIC CLEANING
# -----------------------------
df = df.dropna(
    subset=[
        "area",
        "hour_of_day",
        "day_of_week",
        "promised_duration_minutes",
        "is_late"
    ]
)

# -----------------------------
# FEATURES & TARGET
# -----------------------------
X = df[
    ["area", "hour_of_day", "day_of_week", "promised_duration_minutes"]
].copy()

y = df["is_late"].astype(int)

# -----------------------------
# TRAIN / TEST SPLIT
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.22,
    random_state=42,
    stratify=y
)

# -----------------------------
# PREPROCESSING
# -----------------------------
pre = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["area"]),
        ("num", "passthrough",
         ["hour_of_day", "day_of_week", "promised_duration_minutes"])
    ]
)

# -----------------------------
# RANDOM FOREST MODEL
# -----------------------------
model = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    min_samples_leaf=20,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

# -----------------------------
# PIPELINE
# -----------------------------
pipe = Pipeline([
    ("pre", pre),
    ("model", model)
])

# -----------------------------
# TRAIN
# -----------------------------
pipe.fit(X_train, y_train)

# -----------------------------
# EVALUATION
# -----------------------------
proba = pipe.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)

print("✅ Delay Risk Model trained")
print("Model: RandomForestClassifier")
print("Features: area, hour_of_day, day_of_week, promised_duration_minutes")

print("\n--- Classification Report ---")
print(classification_report(y_test, pred, digits=3))

print("ROC-AUC:", round(roc_auc_score(y_test, proba), 4))


C:\Users\Tasneem Inayath\AppData\Local\Temp\ipykernel_10868\95758556.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ Delay Risk Model trained
Model: RandomForestClassifier
Features: area, hour_of_day, day_of_week, promised_duration_minutes

--- Classification Report ---
              precision    recall  f1-score   support

           0      0.384     0.522     0.442       418
           1      0.624     0.487     0.547       682

    accuracy                          0.500      1100
   macro avg      0.504     0.504     0.495      1100
weighted avg      0.533     0.500     0.507      1100

ROC-AUC: 0.4928


In [10]:
import joblib
joblib.dump(pipe, "delivery_delay_model.pkl")


['delivery_delay_model.pkl']

In [21]:
import pandas as pd
import psycopg2

# ---------------------------
# DB CONNECTION
# ---------------------------
conn = psycopg2.connect(
    host="localhost",
    database="blinkit_db",
    user="postgres",
    password="2741"
)

# ---------------------------
# LOAD FEEDBACK DATA
# ---------------------------
query = """
SELECT
    feedback_id,
    order_id,
    feedback_text,
    feedback_category,
    sentiment,
    feedback_date
FROM blinkit_customer_feedback
WHERE feedback_text IS NOT NULL
  AND feedback_text <> '';
"""

df = pd.read_sql(query, conn)
conn.close()

print("Total feedback rows:", len(df))
df.head()


Total feedback rows: 5000


C:\Users\Tasneem Inayath\AppData\Local\Temp\ipykernel_10868\3234839245.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,feedback_id,order_id,feedback_text,feedback_category,sentiment,feedback_date
0,2234710,1961864118,"It was okay, nothing special.",Delivery,Neutral,2024-07-17
1,5450964,1549769649,The order was incorrect.,App Experience,Negative,2024-05-28
2,482108,9185164487,"It was okay, nothing special.",App Experience,Neutral,2024-09-23
3,4823104,9644738826,The product met my expectations.,App Experience,Neutral,2023-11-24
4,3537464,5427684290,Product was damaged during delivery.,Delivery,Negative,2023-11-20


In [22]:
# ---------------------------
# BASIC TEXT CLEANING
# ---------------------------
df["clean_text"] = (
    df["feedback_text"]
    .str.lower()
    .str.strip()
)

# Remove very short feedback (like "ok", "good")
df = df[df["clean_text"].str.len() > 10]

print("Clean feedback rows:", len(df))
df[["feedback_text", "clean_text"]].head()


Clean feedback rows: 5000


,feedback_text,clean_text
0,"It was okay, nothing special.","it was okay, nothing special."
1,The order was incorrect.,the order was incorrect.
2,"It was okay, nothing special.","it was okay, nothing special."
3,The product met my expectations.,the product met my expectations.
4,Product was damaged during delivery.,product was damaged during delivery.


In [23]:
# Remove exact duplicate feedback texts
df_unique = df.drop_duplicates(subset=["clean_text"]).reset_index(drop=True)

print("Before:", len(df))
print("After :", len(df_unique))


Before: 5000
After : 25


In [25]:
from sentence_transformers import SentenceTransformer
import numpy as np

# ---------------------------
# LOAD EMBEDDING MODEL
# ---------------------------
model = SentenceTransformer("all-MiniLM-L6-v2")

# ---------------------------
# CREATE EMBEDDINGS
# ---------------------------
texts = df_unique["clean_text"].tolist()

embeddings = model.encode(
    texts,
    show_progress_bar=True
)

print("Embeddings shape:", embeddings.shape)


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.40it/s]

Embeddings shape: (25, 384)


In [15]:
import pickle

# Save vectors
with open("C:/Projects/final project/data/feedback_vectors.pkl", "wb") as f:
    pickle.dump(embeddings, f)

# Save metadata (to map back to text)
df_unique[[
    "feedback_id",
    "order_id",
    "clean_text",
    "feedback_category",
    "sentiment",
    "feedback_date"
]].to_pickle("feedback_metadata.pkl")

print("✅ Feedback vectors and metadata saved")


NameError: name 'df_unique' is not defined

In [16]:
import pickle
import pandas as pd
import numpy as np

# Load vectors
with open("C:/Projects/final project/data/feedback_vectors.pkl", "rb") as f:
    embeddings = pickle.load(f)

# Load metadata
metadata = pd.read_pickle("C:/Projects/final project/data/feedback_metadata.pkl")
embeddings = np.array(embeddings)

print(embeddings.shape)
print(metadata.head())


(25, 384)
   feedback_id    order_id                            clean_text  \
0      2234710  1961864118         it was okay, nothing special.   
1      5450964  1549769649              the order was incorrect.   
2      4823104  9644738826      the product met my expectations.   
3      3537464  5427684290  product was damaged during delivery.   
4       230696  4898355547                   highly recommended!   

  feedback_category sentiment feedback_date  
0          Delivery   Neutral    2024-07-17  
1    App Experience  Negative    2024-05-28  
2    App Experience   Neutral    2023-11-24  
3          Delivery  Negative    2023-11-20  
4   Product Quality  Positive    2023-04-16  


In [17]:
user_question = "Why are customers unhappy with fruit orders?"


In [18]:
from sentence_transformers import SentenceTransformer

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

query_vector = embed_model.encode([user_question])


In [21]:
from sklearn.metrics.pairwise import cosine_similarity

# Compare question vector with all feedback vectors
similarity_scores = cosine_similarity(query_vector, embeddings)[0]

SIMILARITY_THRESHOLD = 0.25  # important

top_indices = similarity_scores.argsort()[::-1]

relevant_indices = [
    i for i in top_indices
    if similarity_scores[i] >= SIMILARITY_THRESHOLD
][:5]


In [22]:
for i, row in retrieved_feedbacks.iterrows():
    print("----")
    print(row["clean_text"])


----
delivery was fine, but the product could be better.
----
delivery was late and i was unhappy.
----
customer service was not helpful.
----
the packaging was poor.
----
items were missing from my order.


In [23]:
if len(relevant_indices) == 0:
    answer = "I don’t have enough customer feedback data to answer this question."
    print(answer)
    exit()


In [24]:
retrieved_feedbacks = metadata.iloc[relevant_indices]

context = "\n".join(
    f"- {text}"
    for text in retrieved_feedbacks["clean_text"].tolist()
)


In [25]:
prompt = f"""
You are a senior business analyst at Blinkit.

RULES:
- Answer ONLY using the complaints below.
- If complaints do not explain the question, say:
  "Insufficient customer feedback to determine root cause."

Customer complaints:
{context}

Question:
{user_question}

Provide a clear root-cause analysis in business terms.
"""


In [26]:

from groq import Groq
import os

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.2
)

answer = response.choices[0].message.content
print(answer)


Based on the provided customer complaints, I will attempt to identify the root cause of customer unhappiness with fruit orders.

**Root Cause Analysis:**

1. **Insufficient Quality Control**: The complaint about the product being "could be better" suggests that the quality of the fruits delivered is not meeting customer expectations. This could be due to inadequate quality control measures in place, such as inconsistent sourcing, inadequate storage, or poor handling practices.
2. **Inefficient Supply Chain**: The complaint about late delivery suggests that there may be issues with the supply chain, such as inadequate inventory management, inefficient logistics, or poor communication with suppliers. This could lead to delays in receiving fresh fruits, which may not meet customer expectations.
3. **Poor Packaging**: The complaint about poor packaging suggests that the fruits may not be being handled and stored properly, leading to damage or spoilage during transit. This could be due to i